<a href="https://colab.research.google.com/github/neilgautam/APRIORI-ASSOCIATION_RULE_LEARNING-/blob/master/APRIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth

In [2]:
def equipment_loader(file_path,equipments):
      
    equip_dict = {}
    for equipment in equipments:
        print( file_path+equipment)
        equip_data = np.load(file_path+equipment)
        equipment = equipment[:-4]
        equip_data = list(equip_data)
        equip_data = [ x for x in equip_data if x!= '0']
        equip_dict[equipment] = list(equip_data)
    return equip_dict

In [3]:
def DateString():
    date = input("Enter Date : ")
    month = input("Enter Month : ")
    year = input("Enter Year : ")
    Date= [date,month,year]
    Date ='-'.join(Date) 
    return Date

In [4]:
def DayDataExtraction(Data,equipment_list,Date):
    day_data = {}
    for equipment in equipment_list:
        equipment = equipment[:-4]
        e_data = list(Data[equipment])
        e_data = [ x for x in e_data if Date in x]
        day_data[equipment] = e_data
    cleaned_day_data = {}    
    for equipment in day_data.keys():
        if len(day_data[equipment])==0:
            continue
        else:
            cleaned_day_data[equipment] = day_data[equipment]
    no_of_equipment_w = len(cleaned_day_data.keys()) 
#     print("No of equipment working on {%s} are :"%(Date),end = " ")
#     print(no_of_equipment_w)       
    return cleaned_day_data    

In [5]:
def HourDataExtraction(Data,Date,HourTime,equipment_list):
    day_ = DayDataExtraction(Data,equipment_list,Date)
    #print(day_)
    temp_list = [Date,HourTime[:2]]
    temp_time = ' '.join(temp_list)
    hourwise_data = {}
    for equip in day_.keys():
        temp_data = list(Data[equip])
        temp_data = [ x for x in temp_data if temp_time in x]
        if len(temp_data)==0:
            continue
        else:
            hourwise_data[equip] = temp_data
#     print("No of equipments working at %s %s are :"%(Date,HourTime),end =" ")
#     print(len(hourwise_data.keys()))       
    return hourwise_data       

In [6]:
def TimeDataExtraction(day_,Data,Date,Time,equipment_list):
    day_ = day_
    #day_ = DayDataExtraction(Data,equipment_list,Date)
    #print(day_)
    temp_list = [Date,Time]
    temp_time = ' '.join(temp_list)
    _data = {}
    for equip in day_.keys():
        temp_data = list(Data[equip])
        temp_data = [ x for x in temp_data if temp_time in x]
        if len(temp_data)==0:
            continue
        else:
            _data[Time+' '+equip] = temp_data
#     print("No of equipments working at %s %s are :"%(Date,Time),end =" ")
#     print(len(_data.keys()))       
    return list(_data.keys())

In [7]:
def DayTimeGenerator():
    hour = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
    minute = ['00','05']
    for x in range(10,60):
        if x%5==0:
            minute.append(str(x))
    second = '00'
    Time = []
    for hr in hour:
        for min in minute:
            temp = [hr,min,second]
            temp = ':'.join(temp)
            Time.append(temp)
    return Time

In [8]:
def MonthDateGenerator(year ,month,days_in_the_month):
    year = year
    month = month 
    days = days_in_the_month
    day = ['01','02','03','04','05','06','07','08','09']
    for i in range(10,days+1):
        day.append(str(i))
    Dates = []
    for d in day:
        temp = [year,month,d]
        temp = '-'.join(temp)
        Dates.append(temp)
    return Dates        

In [9]:
def DataExtractor(equipment_list,equipments_data,Dates,Time):
    transactions = []
    no_of_eq = len(equipment_list)
    for date in Dates:
        day_ = DayDataExtraction(equipments_data,equipment_list,date)
        for time in Time:
            temp_list = TimeDataExtraction(day_,equipments_data,date,time,equipment_list)
            if len(temp_list)==0:
                continue
            #if len(temp_list)<no_of_eq:
             #   diff = no_of_eq-len(temp_list)
              #  for i in range(1,diff+1):
               #     temp_list.append('nan')
            else:
                transactions.append(temp_list)
    return transactions                

In [10]:
def get_equipment_list(path):
    equipment_list = []
    for item in os.listdir(path):
        if 'channel_' in item and item != "channel_1.dat":
            equipment_list.append(item)
    return equipment_list

In [11]:
def get_equipment_data(path, equipment_list):
    equipments_data = equipment_loader(path, equipment_list)
    return equipments_data

In [12]:
def get_all_dates(time_period_info):
    dates = []
    for item in time_period_info:
        dates.extend(MonthDateGenerator(item[0],item[1],item[2]))
    return dates

In [13]:
def get_all_time_of_day():
    Time = DayTimeGenerator()
    return Time

In [14]:
def get_apriori_input_data(equipment_list,equipments_data,Dates,Time):
    apriori_data = DataExtractor(equipment_list,equipments_data,Dates,Time)
    return apriori_data

In [15]:
def get_support_and_pattern(apriori_data, minimum_support):
    te = TransactionEncoder()
    data = te.fit(apriori_data).transform(apriori_data)
    data = pd.DataFrame(data, columns = te.columns_)
    return apriori(data, min_support = minimum_support, use_colnames = True)

In [5]:
def get_support_and_pattern_fpgrowth(fpgrowth_data, minimum_support):
    te = TransactionEncoder()
    data = te.fit(fpgrowth_data).transform(fpgrowth_data)
    data = pd.DataFrame(data, columns = te.columns_)
    return fpgrowth(data, min_support = minimum_support, use_colnames = True)

In [16]:
time_period_info = [   ['2013','01',31],
                 ['2013','02',28],
                 ['2013','03',31],
                 ['2013','04',30],
                 ['2013','05',31],
                 ['2013','06',30],
                 ['2013','07',31],
                 ['2013','08',31],
                 ['2013','09',30],
                 ['2013','10',31],
                 ['2013','11',30],
                 ['2013','12',31],
              
                 ['2014','01',31],
                 ['2014','02',28],
                 ['2014','03',31],
                 ['2014','04',30],
                 ['2014','05',31],
                 ['2014','06',30],
                 ['2014','07',31],
                 ['2014','08',31],
                 ['2014','09',30],
                 ['2014','10',31],
                 ['2014','11',30],
                 ['2014','12',31],
              
                 ['2015','01',31],
                 ['2015','02',28],
                 ['2015','03',31],
                 ['2015','04',30],
                 ['2015','05',31],
                 ['2015','06',30],
                 ['2015','07',31],
                 ['2015','08',31],
                 ['2015','09',30],
                 ['2015','10',31],
                 ['2015','11',30],
                 ['2015','12',31]]

In [7]:
min_support = 0.02
fpgrowth_data = np.load("./Channel_On_Off_data/House_2/" + "apriori_data.npy", allow_pickle=True)

In [9]:
house_2_30_min_resampled_patterns_fp_growth = get_support_and_pattern_fpgrowth(fpgrowth_data, min_support)

In [21]:
house_2_30_min_resampled_patterns_fp_growth.sort_values(by=['support'], ascending=False)[:20]

,support,itemsets
191,0.021039,"(21:30:00 channel_4, 21:30:00 channel_6)"
32,0.021039,(21:00:00 channel_4)
192,0.021039,"(21:30:00 channel_4, 21:30:00 channel_6, 21:30..."
33,0.021039,(21:00:00 channel_5)
190,0.021039,"(21:30:00 channel_4, 21:30:00 channel_5)"
189,0.021039,"(21:30:00 channel_6, 21:30:00 channel_5)"
188,0.021039,"(21:00:00 channel_5, 21:00:00 channel_4, 21:00..."
187,0.021039,"(21:00:00 channel_4, 21:00:00 channel_6)"
186,0.021039,"(21:00:00 channel_5, 21:00:00 channel_6)"
185,0.021039,"(21:00:00 channel_5, 21:00:00 channel_4)"
